In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import os
from pyspark.sql.functions import col

spark = (SparkSession
             .builder
             .appName("streaming")
             .getOrCreate()
             )

In [ ]:
schema = StructType([
        StructField("Datetime", StringType(), True),
        StructField("MW", DoubleType(), True)
    ])

In [ ]:
!cd data && ls

In [ ]:
streaming = (
            spark.readStream.schema(schema)
            .csv(f'{os.getcwd()}/data/',header=True)
)

In [ ]:
streaming_query = (streaming
                  .withColumn("Datetime",col("Datetime")[0:10])
                  .writeStream
                  .format('csv')
                  .option("path",f'{os.getcwd()}/abc/')
                  .option('checkpointLocation',f'{os.getcwd()}/abc_checkpoint')
                  .outputMode('append')
                  .start())

In [ ]:
streaming_query = (streaming.writeStream
                  .format('console')
                  .outputMode('update')
                  .start())

In [ ]:
streaming_query = (streaming.writeStream
                  .format('socket')
                  .option('host','localhost')
                  .option('port',9999)
                  .outputMode('update')
                  .start())

In [ ]:
streaming_query = (streaming\
                   .withColumn("Datetime",col("Datetime")[0:10])
                   .groupBy("Datetime")\
                   .mean("MW")\
                   .sort(col('DateTime'))
                   .writeStream\
                   .outputMode('complete')
                   .format("console")
                   .start()
                   .awaitTermination()
                  )

In [ ]:
streaming_query